<a href="https://colab.research.google.com/github/Mohammed-Taasir/IR_2023-GROUP-14/blob/assignment-1/IR_Assignment_1_Question_3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp '/content/drive/MyDrive/IR Assignment-1/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [4]:
path = '/content/CSE508_Winter2023_Dataset/'

In [5]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle
import nltk
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
def getListOfFiles(directory):

    # Parameters: directory: type(string)        
    # returns: list of all files in directory with the full path of file
    
    list_of_files = []

    for i in range(len(filenames)):
      fi = "/content/CSE508_Winter2023_Dataset/"+filenames[i]
      list_of_files.append(fi)
    
    return list_of_files

In [8]:
def lowercase(data):

    # Parameters: data: type(string)
    # returns: lowercase of data   
     
    return data.lower()

def perform_word_tokenize(corpus):
  
    # Parameters:corpus: type(string)   
    # returns word-level tokenization of corpus

    return word_tokenize(corpus)

def remove_stopwords_from_tokens(tokens, stopwords_set):
  
    # Parameters: tokens: type(list)
    #             stopwords_set: type(set)
    # returns: tokens without stopwords

    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set] 
    return tokens_sans_stopwords

def remove_punctuation_from_tokens(tokens):

    # Parameters: tokens: type(list)
    # returns: tokens without punctuation

    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    return tokens_sans_punctuation

def remove_blank_space_tokens(tokens):
    
    #Parameters: tokens: type(list)
    #returns: tokens without blank tokens

    tokens_sans_blank_space = [x for x in tokens if x!='']  
    return tokens_sans_blank_space

In [9]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
  
    
    if preprocess_type=='query':
        return word_tokens_sans_blank_tokens                              # if its a query return tokens in the same order
    return sorted(list(dict.fromkeys(word_tokens_sans_blank_tokens)))     # if its a whole document then return tokens in sorted manner

In [10]:
def create_file_dictionary(list_of_files):
    
    # Paramteres: list_of_files: type(string)
    # returns: file_dictionary with integer key and path_of_file as value
  
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

In [11]:
corpus_root='/content/CSE508_Winter2023_Dataset/'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [48]:
# Printing names of all files

filenames=corpus.fileids()
# print(filenames)

In [13]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


In [14]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

In [15]:
stopwords_set = set(stopwords.words('english'))

In [16]:
newfiles=[]

In [17]:
#adding all docs in a list
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

1400


In [18]:
#adding all docs after preprocessing in a list
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  tokens_sans_punctuation = remove_punctuation_from_tokens(tokens_sans_stopwords)
  tokens_sans_blank_space = remove_blank_space_tokens(tokens_sans_punctuation)
  newfiles[i] = tokens_sans_blank_space
print(len(newfiles))

1400


In [19]:
fileData = newfiles

In [20]:
# for i in range(len(newfiles)):
#   print('File : '+str(i+1)+' ', end='')
#   print(newfiles[i])

In [21]:
def create_unigram_inverted_index(file_dictionary, stopwords_set):
    # initialize unigram inverted index
    unigram_inverted_index = {}
    
    # unigram inverted index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for token in doc_tokens:
            if token in unigram_inverted_index:
                unigram_inverted_index[token][0] += 1               # stores doc_frequency
                unigram_inverted_index[token][1].append(doc_ID)     # appending docs in which that term occurs
            else:
                unigram_inverted_index[token] = []
                unigram_inverted_index[token].append(1)
                unigram_inverted_index[token].append([doc_ID])      # initializing if this token appears for first time
    
    # Saving unigram inverted index using pickle module  
    uii_file = open('unigram_inverted_index_pickle_file', 'wb')
    pickle.dump(unigram_inverted_index, uii_file)
    uii_file.close()

    return unigram_inverted_index

stopwords_set = set(stopwords.words('english')) #getting all stopwords

list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')
uni = create_unigram_inverted_index(list_of_files,stopwords_set)

In [22]:
uii_file = open('unigram_inverted_index_pickle_file', 'rb') #loading pickle model
unigram_inverted_index = pickle.load(uii_file)
uii_file.close()

In [23]:
print(len(unigram_inverted_index))

8996


# (QUESTION 3) PHRASE QUERIES

### (i) Bigram inverted index [30 Marks]
1. Create a bigram inverted index (from scratch; No library allowed) of the dataset obtained
from Q1.
2. Use Python’s pickle module to save and load the bigram inverted index.

In [24]:
newlist=[]
newlist=newfiles

for i in range(len(newfiles)):
   for j in range(len(newfiles[i])-1):
       temp1=newfiles[i][j]
       temp2=newfiles[i][j+1]
       temp=temp1+' '+temp2
       newlist[i][j]=temp
   newlist[i].pop()

bigram_inverted_index = {}
for i in range(len(newlist)):
    doc_tokens = newlist[i]
    for token in doc_tokens:
        if token in bigram_inverted_index:
            bigram_inverted_index[token][0] += 1   # stores doc_frequency
            if( i not in bigram_inverted_index[token][1]):
              bigram_inverted_index[token][1].append(i)    # appending docs in which that term occurs
              
        else:
            bigram_inverted_index[token] = []
            bigram_inverted_index[token].append(1)
            bigram_inverted_index[token].append([i])    # initializing if this token appears for first time'

pi_file = open('bigram_inverted_index_pickle_file', 'wb') #saving the model
pickle.dump(bigram_inverted_index, pi_file)
pi_file.close()

In [25]:
bigram_file = open('bigram_inverted_index_pickle_file', 'rb')
bigram_inverted_index = pickle.load(bigram_file)
bigram_file.close()

In [26]:
len(bigram_inverted_index)

85209

In [27]:
uii_file = open('unigram_inverted_index_pickle_file', 'rb')
unigram_inverted_index = pickle.load(uii_file)
uii_file.close()

In [28]:
  # create set of stop words for preprocessing
stopwords_set = set(stopwords.words('english'))

  # Get List of Files in Dataset
list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')

  # create dictionary of file with docID (integer) as key and full_path of file as value
file_dictionary = create_file_dictionary(list_of_files)

In [29]:
def search_bigram(query,q):
          res = len(query.split())
          if(res>5):
            print("bigram invalid query "+str(q)+" -> Length of input sequence has to be <=5\n")
            return
                     
          sanitized_query = preprocess(query, stopwords_set, 'query')

          flag = 0
          if(len(sanitized_query) == 0):
            print("bigram invalid query "+str(q)+" -> you had entered stopwords in your query\n")
            return

          if(len(sanitized_query) == 1):
            print("bigram invalid query "+str(q)+" -> length of your query is 1, for bigram creation length of input query has to be atleast 2\n")
            return

          flag = 0
          for token in sanitized_query:
            if(token not in unigram_inverted_index):
              print("bigram invalid query "+str(q)+" -> given token "+token+" is not present in our unigram_inverted_index hence no corresponding bigram_inverted_index entry will be found.\n")
              return 

          result = []
          for i in range(len(sanitized_query)-1):
            biword = str(sanitized_query[i] + " " + sanitized_query[i+1])
            if(biword not in bigram_inverted_index):
              print("bigram invalid query "+str(q)+" -> biword "+biword+" is not present in our bigram_inverted_index.\n")
              break
            if i==0:
              result = bigram_inverted_index[biword][1]
            else:
              result = intersection(result, bigram_inverted_index[biword][1])
          
          if(len(result)  == 0):
            print("No document found in bigram_inverted_index.\n")
            return
            
          print('Number of documents retrieved for query {} using bigram inverted index : {}'.format(q,len(result)))
          print('Name of documents retrieved for query {} using bigram inverted index : '.format(q), end='')

          output = []
          for res_doc in result:
            output.append(list_of_files[res_doc])
          print(output)

In [50]:
# (bigram_inverted_index)

### (ii) Positional index [35 Marks]
1. Create a positional index (from scratch; No library allowed) of the dataset obtained from
Q1.
2. Use Python’s pickle module to save and load the positional index.

In [30]:
newfiles=[]

In [31]:
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

1400


In [32]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  tokens_sans_punctuation = remove_punctuation_from_tokens(tokens_sans_stopwords)
  tokens_sans_blank_space = remove_blank_space_tokens(tokens_sans_punctuation)
  newfiles[i] = tokens_sans_blank_space
print(len(newfiles))

1400


In [33]:
len(newfiles)

1400

In [34]:
import os
import pickle

def create_positional_index(newfiles):
    positional_index = {}
    for i in range(len(newfiles)):
        tokens = newfiles[i]
        for j, word in enumerate(tokens):
            if(word in positional_index):
                if(i in positional_index[word]):
                    positional_index[word][i].append(j)
                else:
                    positional_index[word][i] = [j]
            else:
                positional_index[word] = {i:[j]}
    
    pi_file = open('positional_index_pickle_file', 'wb')
    pickle.dump(positional_index, pi_file)
    pi_file.close()

    return positional_index

In [35]:
positional_index = create_positional_index(newfiles)
pi_file = open('positional_index_pickle_file', 'rb')
positional_index = pickle.load(pi_file)
pi_file.close()

In [52]:
print(positional_index)

{'experimental': {0: [0, 5], 10: [48], 11: [27], 16: [30], 18: [27], 24: [86], 28: [129], 29: [19], 34: [61], 40: [26], 46: [109], 51: [10], 52: [7], 57: [51], 68: [70], 69: [11], 73: [0], 77: [74], 83: [0, 11], 98: [93, 130], 100: [180], 102: [55], 111: [79], 114: [96], 120: [76], 122: [18, 44], 136: [7], 139: [99], 141: [18], 153: [33], 155: [7], 167: [89], 169: [129], 170: [60], 172: [11, 52], 175: [10], 178: [112, 122], 182: [64], 183: [50], 185: [33, 54], 186: [9], 187: [107], 188: [0], 190: [106], 194: [19, 74, 89], 196: [45, 113], 201: [115], 202: [32], 205: [123, 129], 206: [41, 87], 211: [206], 215: [122], 219: [37], 221: [70], 224: [172], 226: [74], 229: [94], 233: [80, 97, 125, 136], 244: [11], 250: [43], 255: [0, 9], 256: [100], 261: [104], 270: [0, 12], 272: [37], 276: [36], 281: [131], 282: [72], 285: [19], 293: [43], 294: [10], 303: [145], 306: [76], 328: [50, 350], 329: [35], 333: [110, 130], 337: [23], 338: [0], 343: [0, 175], 344: [22], 345: [54, 102], 346: [5], 353: 

In [36]:
def intersection(lst1, lst2): 
    # Use of hybrid method
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

In [37]:
def difference_within_k(pos1,pos2,k):
  for i in pos1:
    for j in pos2:
      if( i<j and abs(j - i) == k):
        return 1;
  return 0;

In [38]:
def search_positional(query,q):
          res = len(query.split())
          if(res>5):
              print("positional invalid query "+str(q)+" -> Length of input sequence has to be <=5\n")
              return
          
          afterPreprocessQuery = preprocess(query, stopwords_set, 'query')
         
          if(len(afterPreprocessQuery) == 0):
            print("positional invalid query "+str(q)+" -> you had entered stopwords in your query\n")
            return
        
          pair_wise_list = []
          pair = ()
          words = afterPreprocessQuery
          
          for token in words:
            if(token not in unigram_inverted_index):
              print("positional invalid query "+str(q)+" -> given token "+token+" is not present in our positional index\n")
              return
          
          if(len(words) == 1):
              documents = unigram_inverted_index[words[0]][1]
              print('Number of documents retrieved for query {} using positional inverted index : {}'.format(q,len(documents)))
              print('Name of documents retrieved for query {} using positional inverted index : '.format(q), end='')

              out = []
              for res_doc in documents:
                out.append(list_of_files[res_doc])
              print(out)
              return
              
        
          first_token =  words[0]
          result = unigram_inverted_index[first_token][1]
          for i in range(1,len(words)):
            token = unigram_inverted_index[words[i]][1]
            result = intersection(result,token)  #finding all the documents that are common in all the tokens

          if(len(result) == 0):
              print("No document found in our positional index.\n")
              return        
        
          for i, word in enumerate(words[:-1]):
            pair = (word, words[i+1])
            pair_wise_list.append(pair)
          temp = (words[0],words[-1])
          pair_wise_list.append(temp)

        
          result_documents = []
          count = 0
          for doc_id in result:
            for pair in pair_wise_list:
              if(pair == pair_wise_list[-1]):
                continue
              position_tk1 = positional_index[pair[0]][doc_id]
              position_tk2 = positional_index[pair[1]][doc_id]

              if(difference_within_k(position_tk1,position_tk2,1)): 
                count = count + 1
            
            k = len(words) - 1
            pair = pair_wise_list[-1]
 
            if(difference_within_k(positional_index[pair[0]][doc_id],positional_index[pair[1]][doc_id],k)): 
              count = count + 1
            if(count == len(pair_wise_list)-1 and len(words) == 2):
              result_documents.append(doc_id)
            if(count == len(pair_wise_list) and len(words)> 2):
              result_documents.append(doc_id)
            count = 0

          print('Number of documents retrieved for query {} using positional inverted index : {}'.format(q,len(result_documents)))
          print('Name of documents retrieved for query {} using positional inverted index : '.format(q), end='')

          output = []
          for res_doc in result_documents:
            output.append(list_of_files[res_doc])
          print(output)


### (iii) Compare and comment on your results using (i) and (ii). [5 Marks]
1. Input Format:<br>
a. The first line contains N denoting the number of queries to execute<br>
b. The next N lines contain phrase queries<br>
2. Output Format:<br>
a. 4N lines consisting of the results in the following format:<br>
&ensp;i. Number of documents retrieved for query X using bigram inverted index<br>
&ensp;ii. Names of documents retrieved for query X using bigram inverted index<br>
&ensp;iii. Number of documents retrieved for query X using positional index<br>
&ensp;iv. Names of documents retrieved for query X using positional index<br>
3. Perform preprocessing steps (from Q1) on the input sequence as well. Assume the length of the input sequence to be <=5.

In [39]:
def search_query():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    N = int(input("Enter number of phrase queries : ")) #input your query
    inputs = []
    for q in range(N):
        input_query = input("Enter query "+str(q+1)+" : ")
        inputs.append(input_query)

    print('\n')

    count = 1
    for query in inputs:
        search_bigram(query,count)
        # print('\n')
        search_positional(query,count)
        count = count +  1
        print('\n')

In [ ]:
a = search_query()

Enter number of phrase queries : 1
Enter query 1 : experimental


bigram invalid query -> length of your query is 1 for bigram creation length of input query has to be atleast 2
Number of documents retrieved for query 1 using positional inverted index : 316
Name of documents retrieved for query 1 using positional inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0001', '/content/CSE508_Winter2023_Dataset/cranfield0011', '/content/CSE508_Winter2023_Dataset/cranfield0012', '/content/CSE508_Winter2023_Dataset/cranfield0017', '/content/CSE508_Winter2023_Dataset/cranfield0019', '/content/CSE508_Winter2023_Dataset/cranfield0025', '/content/CSE508_Winter2023_Dataset/cranfield0029', '/content/CSE508_Winter2023_Dataset/cranfield0030', '/content/CSE508_Winter2023_Dataset/cranfield0035', '/content/CSE508_Winter2023_Dataset/cranfield0041', '/content/CSE508_Winter2023_Dataset/cranfield0047', '/content/CSE508_Winter2023_Dataset/cranfield0052', '/content/CSE508_Winter2023_Dataset/cranfie

In [ ]:
a = search_query()

Enter number of phrase queries : 1
Enter query 1 : downstream locations along the jet


Number of documents retrieved for query 1 using bigram inverted index : 1
Name of documents retrieved for query 1 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0721']
Number of documents retrieved for query 1 using positional inverted index : 0
Name of documents retrieved for query 1 using positional inverted index : []




In [ ]:
a = search_query()

Enter number of phrase queries : 2
Enter query 1 : incompressible friction coefficients
Enter query 2 : friction coefficient


Number of documents retrieved for query 1 using bigram inverted index : 1
Name of documents retrieved for query 1 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0016']
Number of documents retrieved for query 1 using positional inverted index : 1
Name of documents retrieved for query 1 using positional inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0016']


Number of documents retrieved for query 2 using bigram inverted index : 5
Name of documents retrieved for query 2 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0140', '/content/CSE508_Winter2023_Dataset/cranfield0254', '/content/CSE508_Winter2023_Dataset/cranfield0305', '/content/CSE508_Winter2023_Dataset/cranfield0346', '/content/CSE508_Winter2023_Dataset/cranfield0962']
Number of documents retrieved for query 2 using positional inver

In [40]:
a = search_query()

Enter number of phrase queries : 1
Enter query 1 : transition supersonic wind


Number of documents retrieved for query 1 using bigram inverted index : 4
Name of documents retrieved for query 1 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0007', '/content/CSE508_Winter2023_Dataset/cranfield0040', '/content/CSE508_Winter2023_Dataset/cranfield0182', '/content/CSE508_Winter2023_Dataset/cranfield1211']
Number of documents retrieved for query 1 using positional inverted index : 0
Name of documents retrieved for query 1 using positional inverted index : []




In [41]:
a = search_query()

Enter number of phrase queries : 5
Enter query 1 : downstream locations along the jet
Enter query 2 : there are in supersonic aerodynamics
Enter query 3 : transition supersonic wind
Enter query 4 : experimental
Enter query 5 : transition supersonic


Number of documents retrieved for query 1 using bigram inverted index : 1
Name of documents retrieved for query 1 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0721']
Number of documents retrieved for query 1 using positional inverted index : 0
Name of documents retrieved for query 1 using positional inverted index : []


Number of documents retrieved for query 2 using bigram inverted index : 1
Name of documents retrieved for query 2 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0011']
Number of documents retrieved for query 2 using positional inverted index : 1
Name of documents retrieved for query 2 using positional inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield

In [42]:
a = search_query()

Enter number of phrase queries : 5
Enter query 1 : downstream locations along the jet
Enter query 2 : steady of high speed viscous flow past
Enter query 3 : transition supersonic wind
Enter query 4 : experimental
Enter query 5 : transition supersonic


Number of documents retrieved for query 1 using bigram inverted index : 1
Name of documents retrieved for query 1 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0721']
Number of documents retrieved for query 1 using positional inverted index : 0
Name of documents retrieved for query 1 using positional inverted index : []


bigram invalid query 2 -> Length of input sequence has to be <=5

positional invalid query 2 -> Length of input sequence has to be <=5



Number of documents retrieved for query 3 using bigram inverted index : 4
Name of documents retrieved for query 3 using bigram inverted index : ['/content/CSE508_Winter2023_Dataset/cranfield0007', '/content/CSE508_Winter2023_Dataset/cranfield0040', '/cont

In [ ]:
print(positional_index['experimental'])

{0: [0, 5], 10: [48], 11: [27], 16: [30], 18: [27], 24: [86], 28: [129], 29: [19], 34: [61], 40: [26], 46: [109], 51: [10], 52: [7], 57: [51], 68: [70], 69: [11], 73: [0], 77: [74], 83: [0, 11], 98: [93, 130], 100: [180], 102: [55], 111: [79], 114: [96], 120: [76], 122: [18, 44], 136: [7], 139: [99], 141: [18], 153: [33], 155: [7], 167: [89], 169: [129], 170: [60], 172: [11, 52], 175: [10], 178: [112, 122], 182: [64], 183: [50], 185: [33, 54], 186: [9], 187: [107], 188: [0], 190: [106], 194: [19, 74, 89], 196: [45, 113], 201: [115], 202: [32], 205: [123, 129], 206: [41, 87], 211: [206], 215: [122], 219: [37], 221: [70], 224: [172], 226: [74], 229: [94], 233: [80, 97, 125, 136], 244: [11], 250: [43], 255: [0, 9], 256: [100], 261: [104], 270: [0, 12], 272: [37], 276: [36], 281: [131], 282: [72], 285: [19], 293: [43], 294: [10], 303: [145], 306: [76], 328: [50, 350], 329: [35], 333: [110, 130], 337: [23], 338: [0], 343: [0, 175], 344: [22], 345: [54, 102], 346: [5], 353: [55], 359: [37], 

In [ ]:
print(positional_index['investigation'])

{0: [1], 7: [58], 8: [11, 175], 18: [0], 28: [130], 29: [1], 43: [83], 44: [0], 49: [0, 10], 55: [42], 72: [0], 73: [21], 77: [7], 78: [7], 79: [14], 81: [1, 97], 83: [1, 12], 88: [0, 9], 89: [6], 125: [0, 5], 127: [9], 128: [0], 134: [25], 164: [137], 169: [130], 172: [12], 173: [0, 16], 175: [11], 178: [9, 84], 183: [4], 186: [0], 187: [9, 76], 188: [1], 196: [16], 197: [0, 9, 33], 201: [49], 204: [13], 206: [8], 211: [31], 212: [4], 213: [10], 215: [4, 38], 221: [10, 68], 224: [84], 227: [9], 242: [0, 17], 244: [12], 245: [12], 250: [44], 251: [0], 259: [18], 264: [36], 279: [28], 293: [0], 311: [14], 338: [24], 341: [96], 364: [24], 371: [1, 12, 54], 373: [0], 389: [18], 422: [1], 426: [34], 432: [16], 433: [11, 65], 435: [8], 441: [14, 66, 69], 461: [68], 462: [4], 473: [5], 496: [2], 504: [10], 512: [34], 521: [11, 39], 526: [26], 548: [25, 38], 551: [99], 565: [0], 566: [81], 568: [1], 570: [11], 633: [17], 634: [12], 635: [14, 112], 637: [13], 642: [0], 650: [35], 654: [18], 66